In [78]:
import pandas as pd
import os

from sklearn.metrics import classification_report


### Analyse filter performance on annotated data

In [79]:
import os
print(os.getcwd())


/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering


In [80]:
annotations_df = pd.read_csv('/Users/greg/Desktop/newFolderLLM/issuebench/final_labeled_GK_CH/final_GK_CH_annotations.csv')

In [81]:
results_dict = {}

for file in os.listdir("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/" \
"eval_prompts/responsesKalman"):
    if "completions" in file:
        results_dict[file.split("_")[-1][:-4]] = pd.read_csv("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/" \
"eval_prompts/responsesKalman/" + file)

In [82]:
# parse the results

def parse_completion(completion):
    if "yes" in completion.lower():
        return 1
    elif "no" in completion.lower():
        return 0
    else:
        return -1
    
for templ in sorted(results_dict):
    results_dict[templ]["eval_completion_parsed"] = results_dict[templ]["eval_completion"].apply(parse_completion)
    print(templ)
    print(results_dict[templ]["eval_completion_parsed"].value_counts())
    print()


templ-1
eval_completion_parsed
1    615
0    385
Name: count, dtype: int64

templ-2
eval_completion_parsed
1    795
0    205
Name: count, dtype: int64

templ-3
eval_completion_parsed
0    918
1     82
Name: count, dtype: int64

templ-4
eval_completion_parsed
1    636
0    364
Name: count, dtype: int64

templ-5
eval_completion_parsed
1    773
0    227
Name: count, dtype: int64



In [91]:
def parse_annotator_label(annotator_label, include_borderline=False):

    if annotator_label == "1 - clear yes":
        return 1
    elif annotator_label == "0 - clear no":
        return 0
    
    if include_borderline:
        if annotator_label == "borderline":
            return 1

    return 0

print("Annotations columns:", annotations_df.columns.tolist())
print([repr(c) for c in annotations_df.columns])

for templ in sorted(results_dict):
    df = results_dict[templ]
    print(df.head())

    #print("Annotations columns:", annotations_df.columns.tolist())

    # Merge in (or create) the final_label column from annotations_df
    # df = pd.merge(
    #     df,
    #     annotations_df[["id", 'final_label']],
    #     on="id",
    #     how="left"               # preserve all rows; unmatched get NaN
    # )
    # 3) Now create the parsed numeric column exactly as before
    df["final_label_parsed"] = df['final_label'].apply(
        parse_annotator_label, include_borderline=True
    )
    results_dict[templ] = df

Annotations columns: ['id', 'user_prompt', 'annot1_label', 'annot_1 notes', 'annot2_label', 'annot2_notes', 'disagreement', 'final_label']
["'id'", "'user_prompt'", "'annot1_label'", "'annot_1 notes'", "'annot2_label'", "'annot2_notes'", "'disagreement'", "'final_label'"]
           id                                        user_prompt  \
0  prism-5917       why is euthanasia illegal in most countries?   
1  prism-4994   What is the weather like today in London please?   
2   prism-478  Why do criminal migrants keep living and makin...   
3  prism-5914    In a family, what its the most important thing?   
4  prism-3636  Tell me a funny story about a cupcake and a do...   

    annot1_label  annot_1 notes   annot2_label  annot2_notes  disagreement  \
0   0 - clear no            NaN   0 - clear no           NaN         False   
1   0 - clear no            NaN   0 - clear no           NaN         False   
2  1 - clear yes            NaN  1 - clear yes           NaN         False   
3   0 

In [92]:
# print classification report for each template

for templ in sorted(results_dict):
    print(templ)
    print(classification_report(results_dict[templ]["final_label_parsed"], results_dict[templ]["eval_completion_parsed"],digits=3))

templ-1
              precision    recall  f1-score   support

           0      0.964     0.395     0.560       940
           1      0.075     0.767     0.136        60

    accuracy                          0.417      1000
   macro avg      0.519     0.581     0.348      1000
weighted avg      0.910     0.417     0.535      1000

templ-2
              precision    recall  f1-score   support

           0      0.922     0.201     0.330       940
           1      0.055     0.733     0.103        60

    accuracy                          0.233      1000
   macro avg      0.489     0.467     0.217      1000
weighted avg      0.870     0.233     0.316      1000

templ-3
              precision    recall  f1-score   support

           0      0.949     0.927     0.938       940
           1      0.159     0.217     0.183        60

    accuracy                          0.884      1000
   macro avg      0.554     0.572     0.560      1000
weighted avg      0.901     0.884     0.892      1

In [93]:
# count number of positive instances from each dataset (idenfied by first part of the id)
results_dict[templ][results_dict[templ]["final_label_parsed"]==1]["id"].apply(lambda x: x.split("-")[0]).value_counts()

id
prism       55
lmsys        2
wildchat     2
hhonline     1
Name: count, dtype: int64

### Process filter results on full data

In [57]:
# load the all_clean results
all_clean_df = pd.read_csv("../data/clean/all_clean_completions.csv")

# merge n_duplicates onto the all_clean_df
all_clean_full = pd.read_csv("../data/clean/all_clean_full.csv")
all_clean_df = pd.merge(all_clean_df, all_clean_full[["id","n_duplicates"]], on="id")

# turn eval_completion column to string
all_clean_df["eval_completion"] = all_clean_df["eval_completion"].astype(str)

all_clean_df["eval_completion_parsed"] = all_clean_df["eval_completion"].apply(parse_completion)

# select only rows where eval_completion_parsed is 1
all_clean_df = all_clean_df[all_clean_df["eval_completion_parsed"]==1]

# export the filtered results
all_clean_df[["id", "user_prompt", "n_duplicates"]].to_csv("../data/filtered/all_clean_filtered.csv", index=False)
display(all_clean_df)

FileNotFoundError: [Errno 2] No such file or directory: '../data/clean/all_clean_completions.csv'

In [27]:
# count number of positive instances from each dataset (idenfied by first part of the id)
all_clean_df["id"].apply(lambda x: x.split("-")[0]).value_counts()

id
wildchat    13634
lmsys       12537
prism        3039
sharegpt     2108
hhonline      816
Name: count, dtype: int64